In [ ]:
!pip install pymongo

In [3]:
import pymongo
import re
import requests
import pandas as pd

In [4]:
cli = pymongo.MongoClient(host='34.210.97.79')

In [5]:
wiki_mongo_database = cli.wiki_mongo_database
wiki_mongo_collection = cli.wiki_mongo_database.wiki_mongo_collection
wiki_mongo_collection.count()

5665

In [6]:
df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_mongo_collection.find({})))

In [90]:
df.shape

(5665, 6)

In [59]:
df.sample(10)

,_id,category,clean_content,content,pageid,title
4237,591ca0d8ad28cdd96d81a27d,Business intelligence,otto lucien anton tony spaeth jr february sain...,"Otto Lucien Anton ""Tony"" Spaeth', Jr. (Februar...",25718240,Tony Spaeth
117,591b9cedad28cdd96d814669,machine learning,applied statistic canonical correspondence ana...,"In applied statistics, canonical correspondenc...",40892755,Canonical correspondence analysis
1243,591c77c6ad28cdd96d81792f,Geographic information systems,chameleon open source distribute highly config...,"Chameleon is an open source, distributed, high...",6933277,Chameleon (GIS)
5313,59207268ad28cdd96d85086a,Yoga,haidakhan babaji simply call babaji bhole baba...,"Haidakhan Babaji, simply called ""Babaji"" or Bh...",4320585,Haidakhan Babaji
2868,591c976aad28cdd96d819128,Petroleum,teppco partner lp fortune company base houston...,TEPPCO Partners LP was a Fortune 300 company b...,4774185,TEPPCO Partners
2644,591c96c1ad28cdd96d818f0f,Petroleum,peter gray bear american psychologist currentl...,Peter Gray (born 1944) is an American psycholo...,4791310,Peter Gray (psychologist)
3069,591c97fdad28cdd96d819308,Petroleum,zena werb bear professor vice chair anatomy un...,Zena Werb (born 1945) is a professor and the V...,47149073,Zena Werb
5100,5920708cad28cdd96d85063e,Taiwan,baron samejima kazunori june october admiral e...,"Baron Samejima Kazunori (鮫島員規, 14 June 1845 – ...",42714767,Samejima Kazunori
4417,5920049ead28cdd96d84e769,Taiwan,national museum history nmh chinese pinyin gu ...,The National Museum of History (NMH; Chinese: ...,17783247,National Museum of History
1600,591c7958ad28cdd96d817dd9,Geographic information systems,terrset idrisi integrate geographic informatio...,TerrSet (formerly IDRISI) is an integrated geo...,11944583,TerrSet


In [7]:
# drop previously cleaned content so I can practice cleaning

df = df.drop('clean_content', axis=1)

In [8]:
df.tail(2)

,_id,category,content,pageid,title
5663,59207398ad28cdd96d850b95,Taiwan,The following outline is provided as an overvi...,16285622,Outline of Taiwan
5664,59207398ad28cdd96d850b97,Taiwan,"Taiwan (/ˌtaɪˈwɑːn/), officially the Republic ...",25734,Taiwan


In [10]:
df['category'].value_counts()

Petroleum                         1414
Business intelligence             1174
machine learning                  1077
Taiwan                             582
Yoga                               494
Geographic information systems     470
Statistical_methods                345
Microsoft Office                   109
Name: category, dtype: int64

In [111]:
def cleaner(text):
    text = text.lower()
    text = re.sub('<.{0,3}>',' ',text)
    text = re.sub('{.*\.*}',' ',text)
    text = re.sub('{.*\+.*}',' ',text)
    text = re.sub('[\W]',' ',text)
    text = re.sub(' [^a] ','',text)
    text = re.sub('[0-9]','',text)
    text = re.sub('\s+',' ',text)
    return text

In [112]:
df['clean_content'] = df['content'].apply(cleaner)

In [113]:
df.head()

,_id,category,content,pageid,title,clean_content
0,591b9c60ad28cdd96d814531,machine learning,(1+ε)-approximate nearest neighbor search is a...,32003319,(1+ε)-approximate nearest neighbor search,ε approximate nearest neighbor search is a spe...
1,591b9c65ad28cdd96d814535,machine learning,ADALINE (Adaptive Linear Neuron or later Adapt...,8220913,ADALINE,adaline adaptive linear neuron or later adapti...
2,591b9c66ad28cdd96d814538,machine learning,"AI@50, formally known as the ""Dartmouth Artifi...",27562454,AI@50,ai formally known as the dartmouth artificial ...
3,591b9c67ad28cdd96d81453a,machine learning,AIVA (Artificial Intelligence Virtual Artist) ...,52642349,AIVA,aiva artificial intelligence virtual artist is...
4,591b9c6aad28cdd96d81453e,machine learning,AIXI ['ai̯k͡siː] is a theoretical mathematical...,30511763,AIXI,aixiaisiː is a theoretical mathematical formal...


### Option 1: keep two categories, machine learning and business intelligence
Since there aren't many articles for some of the categories, I only keep the two largest categories, that are about the same size

In [114]:
df_two_cats = df[(df['category']=='machine learning') | (df['category']=='Business intelligence')]

In [119]:
df_two_cats['category'].unique()

array(['machine learning', 'Business intelligence'], dtype=object)

In [120]:
# label encode category column:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_two_cats['category'] = le.fit_transform(df_two_cats['category'])

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [121]:
df_two_cats['category'].unique()

array([1, 0])

In [122]:
df_two_cats.to_pickle('wiki_data_two_cats.p')

### Option 2: keep all categories

In [117]:
# label encode category column:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['category'] = le.fit_transform(df['category'])

In [ ]:
import bz2
import binascii

original_data = 'This is the original text.'
print 'Original     :', len(original_data), original_data

compressed = bz2.compress(original_data)
print 'Compressed   :', len(compressed), binascii.hexlify(compressed)

decompressed = bz2.decompress(compressed)
print 'Decompressed :', len(decompressed), decompressed

In [124]:
df.to_pickle('wiki_data.p')